In [0]:
import pandas as pd
import numpy as np

# elementos do Bokeh 
from bokeh import events
from bokeh.io import curdoc
from bokeh.models import ColumnDataSource, CustomJS, Slider
from bokeh.plotting import figure, show
from bokeh.models import CategoricalColorMapper, Legend, LegendItem, Div
from bokeh.palettes import Spectral9
from bokeh.models.annotations import Legend, Title
from bokeh.io import output_notebook, output_file, save
from bokeh.layouts import column, row

In [0]:
# Datasets
url_discentes_2019 = "http://dados.ufrn.br/dataset/554c2d41-cfce-4278-93c6-eb9aa49c5d16/resource/a55aef81-e094-4267-8643-f283524e3dd7/download/discentes-2019.csv"
url_cursos = "http://dados.ufrn.br/dataset/02526b96-cf40-4507-90b0-3afe5ddd53e7/resource/a10bc434-9a2d-491a-ae8c-41cf643c35bc/download/cursos-de-graduacao.csv"

url_discentes_2018 = "http://dados.ufrn.br/dataset/554c2d41-cfce-4278-93c6-eb9aa49c5d16/resource/146b749b-b9d0-49b2-b114-ac6cc82a4051/download/discentes-2018.csv"
url_discentes_2017 = "http://dados.ufrn.br/dataset/554c2d41-cfce-4278-93c6-eb9aa49c5d16/resource/dc732572-a51a-4d4a-a39d-2db37cbe5382/download/discentes-2017.csv"
url_discentes_2016 = "http://dados.ufrn.br/dataset/554c2d41-cfce-4278-93c6-eb9aa49c5d16/resource/7d2fa5b3-743f-465f-8450-91719b34a002/download/discentes-2016.csv"
url_discentes_2015 = "http://dados.ufrn.br/dataset/554c2d41-cfce-4278-93c6-eb9aa49c5d16/resource/e2b5b843-4f58-497e-8979-44daf8df8f94/download/discentes-2015.csv"
url_discentes_2014 = "http://dados.ufrn.br/dataset/554c2d41-cfce-4278-93c6-eb9aa49c5d16/resource/6c23a430-9a7c-4d0f-9602-1d5d97d40e6a/download/discentes-2014.csv"
url_discentes_2013 = "http://dados.ufrn.br/dataset/554c2d41-cfce-4278-93c6-eb9aa49c5d16/resource/dba208c2-822f-4e26-adc3-b61d4cb110b6/download/discentes-2013.csv"
url_discentes_2012 = "http://dados.ufrn.br/dataset/554c2d41-cfce-4278-93c6-eb9aa49c5d16/resource/fc283aa9-61a7-4cf0-91fb-c403c0817b48/download/discentes-2012.csv"

# ColunaS
COLUNAS_ALUNO = ['sexo', 'raca', 'tipo_discente', 'status', 'sigla_nivel_ensino', 
                 'matricula', 'id_curso']
COLUNAS_CURSO = ['nome', 'nivel_ensino', 'area_conhecimento', 'id_curso', 
                 'modalidade_educacao']

STATUS_ALUNO = ['ATIVO', 'CONCLUÍDO']

COLUNAS = ['F','M','amarelo','branco','indigeno','negro',
           'nao_informado','pardo', 'remanescente_de_quilombo']

discentes = pd.read_csv(url_discentes_2019, sep=';')
alunos_2019 = (np.isin(discentes.status, STATUS_ALUNO)) & (discentes.sigla_nivel_ensino == 'G')
dados_discentes = discentes.loc[ alunos_2019 , COLUNAS_ALUNO ]

cursos = pd.read_csv(url_cursos, sep=';')
cursos_graduacao = cursos.loc[ cursos.nivel_ensino == 'GRADUAÇÃO',  COLUNAS_CURSO ]
dist_cursos_2019 = cursos_graduacao.set_index('id_curso')

In [0]:
discentes.status.unique()

array(['ATIVO', 'CADASTRADO', 'CANCELADO', 'CONCLUÍDO', 'TRANCADO',
       'ATIVO - FORMANDO'], dtype=object)

Passamos os valores encontrados de volta para o dataset principal

## O que mudou?

Nesta seção, são feitas comparações entre a distribuição de gênero e raça ao longo dos anos.

Primeiro, é preciso fazer uma função que repita todos os processos feitos com o dataset de 2019 para os anos anteriores e una tudo em um dataset só.

In [0]:
def processa_dataset(url_discentes, ano, dataset):
    """ Lê o dataset de um ano, processa as estatisticas e 
    adiciona ao dataset geral
    """
    
    discentes_dataset = pd.read_csv(url_discentes, sep=';')
    alunos = np.isin(discentes_dataset.status, STATUS_ALUNO) & (discentes_dataset.sigla_nivel_ensino == 'G')
    dados_discentes_dataset = discentes_dataset.loc[ alunos , COLUNAS_ALUNO ]
    
    cursos_e_alunos_dt = pd.merge(left=cursos_graduacao, 
                                       right=dados_discentes_dataset, on='id_curso')

    # Contabilizando sexos por curso
    cursos_sexos_dt_count = pd.pivot_table(cursos_e_alunos_dt, values='matricula', index=['id_curso'],                    
              columns=['sexo'], aggfunc=lambda x: len(x), fill_value=0)

    dist_cursos = cursos_graduacao.set_index('id_curso')
    dist_cursos['F'] = cursos_sexos_dt_count['F']
    dist_cursos['M'] = cursos_sexos_dt_count['M']

    # Contabilizando raças por curso
    cursos_racas_count_dataset = pd.pivot_table(cursos_e_alunos_dt, values='matricula', index=['id_curso'],                    
              columns=['raca'], aggfunc=lambda x: len(x), fill_value=0)

    cursos_racas_count_dataset.columns = cursos_racas_count_dataset.columns.str.lower()

    # Corrige nome das colunas geradas para raça
    names_dt = {
        "amarelo (de origem oriental)": "amarelo",
        "indígeno": "indigeno",
        "não informado": "nao_informado",
        "remanescente de quilombo": "remanescente_de_quilombo"
    }
    cursos_racas_count_dataset = cursos_racas_count_dataset.rename(columns=names_dt)
    
    for raca in cursos_racas_count_dataset.columns:
        dist_cursos[raca] = cursos_racas_count_dataset[raca]

    cursos_agrupados_dataset = dist_cursos.fillna(0).groupby('nome')

    # Inicializa dataset que contém apenas as colunas usadas
    dados = pd.DataFrame(columns = ['nome', 'area_conhecimento', 'ano'] + COLUNAS)
    dados['nome'] = cursos_agrupados_dataset.groups.keys()

    # TODO: fazer esse loop de forma pythonica
    for curso in cursos_agrupados_dataset.groups.keys():
        area = cursos_agrupados_dataset.get_group(curso).agg(lambda x:x.value_counts().index[0])
        dados.loc[dados.nome == curso, 'area_conhecimento'] = area.get('area_conhecimento')

        for coluna in COLUNAS:
            try:
                grupo = cursos_agrupados_dataset.get_group(curso)[coluna]
                dados.loc[dados.nome == curso, coluna] = grupo.sum()
            except KeyError:
                dados.loc[dados.nome == curso, coluna]  = 0

    # print(dados.loc[dados.nome == 'TECNOLOGIA DA INFORMAÇÃO', ['F', 'M']])
    # Remove cursos que não possuem alunos
    remover = dados.loc[(dados.F == 0) & (dados.M==0),'nome']
    dados = dados.drop(remover.index)

    dados['mulheres_no_curso(%)'] = dados['F'] / (dados['M'] + dados['F'])
    # print(dados.loc[dados.nome == 'TECNOLOGIA DA INFORMAÇÃO', 'mulheres_no_curso(%)'])

    racas = COLUNAS[2:]
    total_racas = dados[ racas ].sum(axis='columns')
    for raca in racas:
        dados[raca+'_no_curso(%)'] = dados[raca] / total_racas

    # Define o ano
    dados.loc[:,'ano'] = ano

    return dataset.append(dados, ignore_index=True)

Com a função definida, fazemos o tratamento dos alunos entre 2015 e 2019.

In [0]:
dataset = pd.DataFrame()
dataset = processa_dataset(url_discentes_2019, 2019, dataset)
dataset = processa_dataset(url_discentes_2018, 2018, dataset)
dataset = processa_dataset(url_discentes_2017, 2017, dataset)
dataset = processa_dataset(url_discentes_2016, 2016, dataset)
dataset = processa_dataset(url_discentes_2015, 2015, dataset)
dataset = processa_dataset(url_discentes_2014, 2014, dataset)
dataset = processa_dataset(url_discentes_2013, 2013, dataset)
dataset = processa_dataset(url_discentes_2012, 2012, dataset)

In [0]:
dataset.ano.unique()

array([2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012])

### Visualização

O gráfico para mostrar a mudança ao longo dos anos será iterativo, mudando conforme o ano.

Para isso, primeiro definimos uma função que atualiza o gráfico de acordo com o movimento do mouse.

In [0]:
# Definindo os dados para os graficos
source_negros = ColumnDataSource(data={
    'x'       : dataset.loc[dataset.ano == 2019, "negro_no_curso(%)"],
    'y'       : dataset.loc[dataset.ano == 2019, "mulheres_no_curso(%)"],
    'curso'   : dataset.loc[dataset.ano == 2019, 'nome'],
    'area'    : dataset.loc[dataset.ano == 2019, 'area_conhecimento'],
})

source_pardos = ColumnDataSource(data={
    'x'       : dataset.loc[dataset.ano == 2019, "pardo_no_curso(%)"],
    'y'       : dataset.loc[dataset.ano == 2019, "mulheres_no_curso(%)"],
    'curso'   : dataset.loc[dataset.ano == 2019, 'nome'],
    'area'    : dataset.loc[dataset.ano == 2019, 'area_conhecimento'],
})

source_indigenas = ColumnDataSource(data={
    'x'       : dataset.loc[dataset.ano == 2019, "indigeno_no_curso(%)"],
    'y'       : dataset.loc[dataset.ano == 2019, "mulheres_no_curso(%)"],
    'curso'   : dataset.loc[dataset.ano == 2019, 'nome'],
    'area'    : dataset.loc[dataset.ano == 2019, 'area_conhecimento'],
})

source_brancos = ColumnDataSource(data={
    'x'       : dataset.loc[dataset.ano == 2019, "branco_no_curso(%)"],
    'y'       : dataset.loc[dataset.ano == 2019, "mulheres_no_curso(%)"],
    'curso'   : dataset.loc[dataset.ano == 2019, 'nome'],
    'area'    : dataset.loc[dataset.ano == 2019, 'area_conhecimento'],
})

source_amarelos = ColumnDataSource(data={
    'x'       : dataset.loc[dataset.ano == 2019, "amarelo_no_curso(%)"],
    'y'       : dataset.loc[dataset.ano == 2019, "mulheres_no_curso(%)"],
    'curso'   : dataset.loc[dataset.ano == 2019, 'nome'],
    'area'    : dataset.loc[dataset.ano == 2019, 'area_conhecimento'],
})



In [0]:
dataset.loc[dataset.ano == 2015 , "pardo_no_curso(%)"].head(5)

299    0.490196
300    0.444444
301    0.473684
302    0.238095
303    0.514286
Name: pardo_no_curso(%), dtype: object

In [0]:
dataset.loc[dataset.ano == 2019 , "negro_no_curso(%)"].head(5)

0    0.0825688
1            0
2     0.047619
3     0.047619
4    0.0909091
Name: negro_no_curso(%), dtype: object

In [0]:
all_datasets = dict()
all_datasets[2012] = dict()
all_datasets[2013] = dict()
all_datasets[2014] = dict()
all_datasets[2015] = dict()
all_datasets[2016] = dict()
all_datasets[2017] = dict()
all_datasets[2018] = dict()
all_datasets[2019] = dict()

COLUNAS_FINAL = ['mulheres_no_curso(%)', 'amarelo_no_curso(%)','branco_no_curso(%)','indigeno_no_curso(%)','negro_no_curso(%)',
           'pardo_no_curso(%)', 'nome', 'area_conhecimento']
for coluna in COLUNAS_FINAL:
  all_datasets[2012][coluna] = dataset.loc[dataset.ano == 2012, coluna ].values
  all_datasets[2013][coluna] = dataset.loc[dataset.ano == 2013, coluna ].values
  all_datasets[2014][coluna] = dataset.loc[dataset.ano == 2014, coluna ].values
  all_datasets[2015][coluna] = dataset.loc[dataset.ano == 2015, coluna ].values
  all_datasets[2016][coluna] = dataset.loc[dataset.ano == 2016, coluna ].values
  all_datasets[2017][coluna] = dataset.loc[dataset.ano == 2017, coluna ].values
  all_datasets[2018][coluna] = dataset.loc[dataset.ano == 2018, coluna ].values
  all_datasets[2019][coluna] = dataset.loc[dataset.ano == 2019, coluna ].values

In [0]:
def create_legend(plot):
  """Given a plot, put the legend outside the graphic
  """
  legend_items = plot.legend[0].items.copy()
  legend = Legend(items=legend_items, location="center")
  plot.legend.visible = False
  plot.add_layout(legend, 'right')

In [0]:
def create_plot(source, x_label, y_label, color_mapper, raca, tooltips=None):
  plot = figure(title='RELAÇÃO ENTRE RAÇA/COR E GÊNERO NOS CURSOS EM 2019', 
              plot_height = 600, plot_width = 800,
              x_range=(0, 1), x_axis_label=x_label, 
              y_range=(0, 1), y_axis_label=y_label, tooltips=tooltips,
              tools='hover,save')


  # Add the color mapper to the circle glyph
  circ = plot.circle(x='x', y='y', fill_alpha=0.8, source=source,
              color=dict(field='area', transform=color_mapper), legend='area',
             size=10)

  # Coloca a legenda fora do gráfico
  create_legend(plot)
  
  t = Title()
  
  callback = CustomJS(args=dict(source=source, dt=all_datasets, raca = raca, 
                                plot=plot, title=t), code="""
        var data = source.data;
        var f = cb_obj.value
        var x = data['x']
        var y = data['y']
        var curso = data['curso']
        var area = data['area']
        
        source.data['x'] = dt[f][raca+'_no_curso(%)']
        source.data['y'] = dt[f]['mulheres_no_curso(%)']
        source.data['curso'] = dt[f]['nome']
        source.data['area'] = dt[f]['area_conhecimento']
        
        plot.title.text = 'RELAÇÃO ENTRE RAÇA/COR E GÊNERO NOS CURSOS EM ' + f
        
        source.change.emit();
    """)

  slider = Slider(start=2012, end=2019, step=1, value=2019, title='Ano', callback=callback)
  
  return plot, slider

In [0]:
output_notebook()

# Cria lista de todas as áreas de conhecimento
areas_list = dataset.loc[dataset.ano == 2019, 'area_conhecimento'].unique().tolist()

# Inicializa o color mapper para as legendas
color_mapper = CategoricalColorMapper(factors=areas_list, palette=Spectral9)

# Cria plot comparando alunos negros e mulheres
tool_negros = [
    ("curso", "@curso"),
    ("negros", "@x{%0.2f}"),
    ("mulheres", "@y{%0.2f}")
]
plot_negros, slider_negros = create_plot(source_negros, 'Porcentagem de alunos negros no curso', 
           'Porcentagem de mulheres no curso', color_mapper, 'negro', tool_negros)

div = Div(width=400, height=plot_negros.plot_height)

# Cria plot comparando alunos pardos e mulheres
tool_pardos = [
    ("curso", "@curso"),
    ("pardos", "@x{%0.2f}"),
    ("mulheres", "@y{%0.2f}")
]
plot_pardos, slider_pardos = create_plot(source_pardos, 'Porcentagem de alunos pardos no curso', 
           'Porcentagem de mulheres no curso', color_mapper,'pardo', tool_pardos)

# Cria plot comparando alunos indígenas e mulheres
tool_indigenos = [
    ("curso", "@curso"),
    ("indígenos", "@x{%0.2f}"),
    ("mulheres", "@y{%0.2f}")
]
plot_indigenos, slider_indigenos = create_plot(source_indigenas, 'Porcentagem de alunos indígenas no curso', 
           'Porcentagem de mulheres no curso', color_mapper, 'indigeno',tool_indigenos)

# Cria plot comparando alunos brancos e mulheres
tool_brancos = [
    ("curso", "@curso"),
    ("brancos", "@x{%0.2f}"),
    ("mulheres", "@y{%0.2f}")
]
plot_brancos, slider_brancos = create_plot(source_brancos, 'Porcentagem de alunos brancos no curso', 
           'Porcentagem de mulheres no curso', color_mapper, 'branco', tool_brancos)

# Cria plot comparando alunos brancos e mulheres
tool_amarelos = [
    ("curso", "@curso"),
    ("amarelos", "@x{%0.2f}"),
    ("mulheres", "@y{%0.2f}")
]
plot_amarelos, slider_amarelos = create_plot(source_amarelos, 'Porcentagem de alunos de origem oriental no curso', 
           'Porcentagem de mulheres no curso', color_mapper, 'amarelo', tool_amarelos)

layout = column( [column(slider_negros, plot_negros), 
                  column(slider_pardos, plot_pardos), 
                  column(slider_indigenos, plot_indigenos),
                  column(slider_brancos, plot_brancos),
                  column(slider_amarelos, plot_amarelos)
                 ])
## https://bokeh.pydata.org/en/latest/docs/user_guide/interaction/callbacks.html
curdoc().add_root(layout)
show(layout)

output_file("teste1.html")
save(layout)

Loading BokehJS ...

'/content/teste1.html'